In [468]:
import numpy as np
import pandas as pd
import datetime
import time

In [469]:
cust = pd.read_csv('customer_accounts.csv')
acct = pd.read_csv('financials_accounts.csv')
bal = pd.read_csv('financials_balances.csv',parse_dates = ['accrual_date'])
txn = pd.read_csv('financials_transactions.csv',parse_dates = ['accrual_date'])

In [470]:
acct.head()

,id,customer_account_id,name
0,finacc_ctzrz94k00051v354687jg1,aacc_c10cwqghq00001owfyvwq126,Bank account 1
1,finacc_ctzrz8vz00031v3wyouo0dk,aacc_c10cwqghq00001owfyvwq126,Bank account 2
2,finacc_ctzrz91d00041v3h6cklh2s,aacc_c10cwqghq00001owfyvwq126,Bank account 3
3,finacc_ctzrz98x00071v3k3hz7qls,aacc_c10cwqghq00001owfyvwq126,Bank account 4
4,finacc_ctzrz9aw00081v3ghgxf64s,aacc_c10cwqghq00001owfyvwq126,Bank account 5


In [471]:
acct = acct.merge(cust, how='left',left_on='customer_account_id',right_on='id')
#aading customer info from customer_account table on account id table
acct.head()

,id_x,customer_account_id,name_x,id_y,name_y
0,finacc_ctzrz94k00051v354687jg1,aacc_c10cwqghq00001owfyvwq126,Bank account 1,aacc_c10cwqghq00001owfyvwq126,Customer account 1
1,finacc_ctzrz8vz00031v3wyouo0dk,aacc_c10cwqghq00001owfyvwq126,Bank account 2,aacc_c10cwqghq00001owfyvwq126,Customer account 1
2,finacc_ctzrz91d00041v3h6cklh2s,aacc_c10cwqghq00001owfyvwq126,Bank account 3,aacc_c10cwqghq00001owfyvwq126,Customer account 1
3,finacc_ctzrz98x00071v3k3hz7qls,aacc_c10cwqghq00001owfyvwq126,Bank account 4,aacc_c10cwqghq00001owfyvwq126,Customer account 1
4,finacc_ctzrz9aw00081v3ghgxf64s,aacc_c10cwqghq00001owfyvwq126,Bank account 5,aacc_c10cwqghq00001owfyvwq126,Customer account 1


* Taking customer_accounts table as masterlist. Only calculating daily balance info for customers in this table.

In [472]:
acct=acct.drop('id_y',1)

In [473]:
acct = acct.dropna(subset=['name_y'])
#dropping accounts for customers who are not in customer_accounts table

In [474]:
acct.columns=['account_id','customer_account_id','bank_name','customer_name']

In [475]:
bal = bal.merge(acct[['account_id','customer_account_id']],how='left',on='account_id')
#adding customer account id to balance table

In [476]:
bal = bal.dropna(subset=['customer_account_id'])
#dropping balance info for accounts not in customer_accounts table

* One customer can have multiple accounts.
* Balance per customer is needed, hence grouping by and summing balance and transaction info for each customer-date combination

In [477]:
balGrp = bal.groupby(['customer_account_id','accrual_date'])

In [478]:
dailyBal = balGrp['amount'].sum().reset_index()
# summing balance info for each customer-date 

In [479]:
txn = txn.merge(acct[['account_id','customer_account_id']],how='left',on='account_id')

In [480]:
txn = txn.dropna(subset=['customer_account_id'])

In [481]:
txnGrp = txn.groupby(['customer_account_id','accrual_date'])

In [482]:
dailyTxn = txnGrp['amount'].sum().reset_index()
# summing transaction info for each customer-date 

In [483]:
dailyTxn['combo'] = dailyTxn.customer_account_id+dailyTxn.accrual_date.astype('str') 
dailyBal['combo'] = dailyBal.customer_account_id+dailyBal.accrual_date.astype('str') 
# concatinating account id and date to make it easier to look up 

In [484]:
customerList = dailyTxn.customer_account_id.append(dailyBal.customer_account_id)
#Only accounting for customer id s that have at least one balance reading or one transaction

In [485]:
customerList = customerList.unique()

#### Creating Table with each customer's earliest transaction or balance date

In [486]:
minMaxDate = pd.DataFrame(data=customerList,columns=['customer_account_id'])

In [487]:
def minBalanceDateFn(x):
    if x not in np.array(dailyBal.customer_account_id):
        a = dailyTxn[dailyTxn.customer_account_id==x]['accrual_date'].nsmallest(1)
        return (a.iloc[0])
    if x not in np.array(dailyTxn.customer_account_id):
        b = dailyBal[dailyBal.customer_account_id==x]['accrual_date'].nsmallest(1)
        return (b.iloc[0])
    a = dailyBal[dailyBal.customer_account_id==x]['accrual_date'].nsmallest(1)
    b = dailyTxn[dailyTxn.customer_account_id==x]['accrual_date'].nsmallest(1)
    return (min(a.iloc[0],b.iloc[0]))

In [488]:
bal.sort_values(by='accrual_date').tail()

,id,account_id,amount,accrual_date,customer_account_id
193071,finblce_ckkrwpd6natl7pvmj5td9or,finacc_cvjmalvl001gf1vwbf5rwjrw,1569009,2018-09-22,cuacc_cvjghnrha00l21wt2kq3dnr8
193070,finblce_ckkp6cpznan47pvda5stsox,finacc_cvjmalcyb01gd1vw6gcputju,135002424,2018-09-22,cuacc_cvjghnrha00l21wt2kq3dnr8
191111,finblce_ci38bfte6xs47pv4ygemji8,finacc_c8b3tx2wd03fi1u9r8ioium3,17341733,2018-09-22,cuacc_c70dmcdk204fn11m6b2qxg01
191988,finblce_cj3o5r0fdiwa7pvsjibnjac,finacc_cjbdvkbhu006f1wvswp8ybyr,6662014,2018-09-22,cuacc_c62m5io280cg81xb532uctag
191746,finblce_citqvyrjbnf97pv89g9gn0q,finacc_cg192y596000l1umlxludzzy,85579577,2018-09-22,cuacc_ccxw7gf6u00c913795io7ct0


* Max Date is taken to be by default the last known transaction/balance date for any account; which is 2018-09-22

In [489]:
endDate = bal.sort_values(by='accrual_date').tail().iloc[1,3]
#assigning last date to a variable 

In [490]:
minMaxDate['minDate']=minMaxDate.customer_account_id.apply(minBalanceDateFn)

In [491]:
minMaxDate['combo'] = minMaxDate.customer_account_id+minMaxDate.minDate.astype('str') 

In [492]:
minMaxDate['flag']=1

#### Creating a template called dailyBalance with each customer ID and Date for which balance info will be populated

In [493]:
dailyBalance = pd.DataFrame(columns = ['date','customer_account_id'])

In [494]:
for i in range(len(minMaxDate)):
    dateRange = pd.date_range(start=minMaxDate.get_value(i,'minDate'),end=endDate)
    tempDf = pd.DataFrame(data = dateRange,columns = ['date'])
    tempDf['customer_account_id'] = minMaxDate.get_value(i,'customer_account_id')
    dailyBalance = dailyBalance.append(tempDf)

#.get_value and .set_value are used since they are considerably faster than the alternatives (.at or .loc)

c:\users\yash.khullar\appdata\local\continuum\anaconda3\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app
c:\users\yash.khullar\appdata\local\continuum\anaconda3\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


* Balance info will be reported from the account's first day of noted activity (minimum of first balance date and first transaction date) to a standard end date of 2018-09-22. 
* Above template - dailyBalance contains all possible customer id-date combinations for which balance information is needed

In [495]:
dailyBalance['combo'] = dailyBalance.customer_account_id+dailyBalance.date.astype('str') 

In [496]:
dailyBalance = dailyBalance.merge(dailyBal[['combo','amount']], how='left', on = 'combo')
#adding exisiting balance info on to dailyBalance

In [497]:
dailyBalance.rename({'amount': 'balance'}, axis=1, inplace=True)

In [498]:
dailyBalance = dailyBalance.merge(dailyTxn[['combo','amount']], how='left', on = 'combo')
#adding exisiting transaction info on to dailyBalance

In [499]:
dailyBalance.rename({'amount': 'txn'}, axis=1, inplace=True)

In [500]:
dailyBalance.txn.fillna(0,inplace=True)
#none transaction value is assumed to be 0

In [501]:
dailyBalance = dailyBalance.merge(minMaxDate[['combo','flag']], how='left',on='combo')
# adding a flag which signifies if a certain customer-date combo is the first noted activity of that customer

In [502]:
dailyBalance.sort_values(by=['customer_account_id','date'],inplace=True)

In [503]:
for i in range(len(dailyBalance)):
    if pd.isna(dailyBalance.get_value(i,'balance'))==False:
        continue
    else:
        if dailyBalance.get_value(i,'flag')==1:
            dailyBalance.set_value(i,'balance',dailyBalance.get_value(i,'txn'))
        else:
            dailyBalance.set_value(i,'balance',(dailyBalance.get_value(i-1,'balance')+dailyBalance.get_value(i,'txn')))
#.get_value and .set_value are used since they are considerably faster than the alternatives (.at or .loc)

c:\users\yash.khullar\appdata\local\continuum\anaconda3\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app
c:\users\yash.khullar\appdata\local\continuum\anaconda3\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\yash.khullar\appdata\local\continuum\anaconda3\lib\site-packages\ipykernel\__main__.py:6: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\yash.khullar\appdata\local\continuum\anaconda3\lib\site-packages\ipykernel\__main__.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
c:\users\yash.khullar\appdata\local\continuum\anaconda3\lib\sit

#### Logic in words
* If balance info already exists, that value is taken to be final (regardless of any prior balance/transaction info)
* If balance info does not exist, and it is the customer's first date of account activity, then balance is equal to the day's transaction value. Unless first balance info is provided, first day opening balance is assumed to be 0
* If balance info does not exist, and it is NOT the customer's first day of account activity, then balance is equal to the day's transaction value added to the previous day's account balance (day1 balance = day1 transaction + day0 balance)

In [506]:
result = dailyBalance.drop(['combo','txn','flag'],1)

In [508]:
result.to_csv('FinalResult.csv')